# BLEU-n 评估


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
import numpy as np

# 加载预训练模型和 tokenizer
model_name = "C:/Users/Admin/Desktop/model/qwen/Qwen2___5-0___5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 确保 tokenizer 定义了填充标记
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# 加载评估数据集
data_files = {"test": "C:/Users/Admin/Desktop/Data/huatuo_train.json"}
dataset = load_dataset("json", data_files=data_files, split="test")

c:\Users\Admin\.conda\envs\bloom_ft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = "介绍你自己"
messages = [
    {
        "role": "system",
        "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
    },
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024,
)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

In [ ]:
# 提取对话中的问题和参考值
def extract_conversations(dataset):
    questions = []
    references = []
    for item in dataset:
        for conversation in item["conversations"]:
            if conversation["from"] == "human":
                questions.append(conversation["value"])
            elif conversation["from"] == "gpt":
                references.append(conversation["value"])
    return questions, references


questions, references = extract_conversations(dataset)


def predict_responses(questions):
    responses = []
    for question in questions:
        messages = [
            {
                "role": "system",
                "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
            },
            {"role": "user", "content": question},
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=1024,
        )
        generated_ids = [
            output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        responses.append(response)
    return responses


predictions = predict_responses(questions)


# 打印一些示例的参考答案和预测结果
for i in range(5):
    print(f"Question: {questions[i]}")
    print(f"Reference: {references[i]}")
    print(f"Prediction: {predictions[i]}")
    print()

c:\Users\Admin\.conda\envs\bloom_ft\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Question: 曲匹地尔片的用法用量
Reference: 注意：同种药品可由于不同的包装规格有不同的用法或用量。本文只供参考。如果不确定，请参看药品随带的说明书或向医生询问。口服。一次50～100mg（1-2片），3次/日，或遵医嘱。
Prediction: 曲匹地尔是一种用于治疗慢性乙型肝炎、丙型肝炎和乙肝病毒感染的药物。在使用曲匹地尔时，需要遵循医生的建议进行用药。以下是关于曲匹地尔的一些常见用法用量信息：

1. **成人常用量**：通常推荐成人每日口服250mg（即1片），分两次服用。

2. **儿童用量**：对于儿童，剂量可能会根据年龄、体重等因素有所不同。一般情况下，儿童的剂量会比成人低一些，并且需要在医生指导下调整。

3. **注意事项**：
   - 请严格按照医嘱或药品说明书上的指示来服用。
   - 如果出现任何不适或副作用，请立即停止服药并咨询医生。
   - 在开始或调整治疗方案后，应定期复查肝功能和其他相关指标。

4. **特殊人群**：孕妇、哺乳期妇女以及有其他健康问题的人群，在使用曲匹地尔前，应向医生详细说明自己的情况。

请注意，以上信息仅供参考，具体用药应在专业医疗人员的指导下进行。如果您有任何疑问或担忧，最好直接与您的医生沟通以获得个性化的指导。

Question: 三期梅毒多久能治愈吗
Reference: 梅毒是一种进展十分缓慢的疾病，包括早期梅毒，晚期梅毒等等，一般来说，梅毒病程超过两年的就是晚期梅毒了，而晚期梅毒的治疗难度是比较大的，三期梅毒能完全治愈吗?该如何进行治疗?一、对于三期梅毒的治疗，中西各有妙招，希望患者在医生的指导下选择适合自己的治疗方法。被确诊为三期梅毒的患者很关心这个问题，梅毒需要讲究科学的治疗方法，三期梅毒患者如果有一个良好的心态，积极配合医生治疗，还有很有可能治愈的，具体治疗方法我们看看下文的介绍。二、三期梅毒又称晚期梅毒，是因梅毒初期没有进行治疗或者治疗不彻底所造成。好发于40-50岁之间。该期梅毒不仅局限于皮肤和粘膜，也可侵犯任何内脏器官和组织。三期梅毒可发生在感染后2年以上，一般多发于感染后3～4年。病程漫长，可持续10～30年。未经治愈的二期梅毒中约有1/3的病人可发展为晚期活动性梅毒;另有一部分患者不出现晚期梅毒症状，只是梅毒血清反应持续阳性，为晚期潜伏梅毒;也有一部分患者可以

In [3]:
import jieba

# 定义平滑函数
smooth = SmoothingFunction().method4

for ref, pred in zip(references, predictions):
    ref_tokens = list(jieba.cut(ref))
    pred_tokens = list(jieba.cut(pred))
    score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth)
    print(score)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.739 seconds.
Prefix dict has been built successfully.


0.01171631283260721
0.03333193677292866
0.014933298795326507
0.002003876498020886
0.017318967314310183
0.023023495078963882
0.002732883416558408
0.01650567956432504
0.02050946499312018
0.01669094818873859
0.014492363018640464
0.0074263203020694555
0.022158708904137382
0.0023242524089648136
0.009909722051549242
0.00990552193457054
